# Establishing a connection with Sql Server

In [1]:
import pyodbc

try:
    con=pyodbc.connect(
        'DRIVER={Odbc Driver 17 for SQL Server};'
        'SERVER=DESKTOP-4IVU8N4\\MSSQLSERVER1;'
        'UID=sa;'
        'PWD=abcd;',
        autocommit=True
    )
except Exception as e:
        print("Connect it properly",e)

# Creation of Database using Sql Server

In [2]:
try:
    cursor=con.cursor()
    demo = 'SCD'
    create_db_query=f"Create database {demo};"
    cursor.execute(create_db_query)
    print("Hoorah, Database Created!!")
except Exception as e:
    print('Work more on Database Connection')

Work more on Database Connection


# Using SQLITE3, it is used to create a database within the venv for temporary purpose


In [3]:
# import sqlite3

# try:
#     con=sqlite3.connect("SCD.db")
#     print("connected successfully")
# except Exception as e:
#     print("not connected")

# Connecting with Created Database

In [4]:
from sqlalchemy import create_engine

try:
    engine = create_engine('mssql+pyodbc://@DESKTOP-4IVU8N4\MSSQLSERVER1/SCD?driver=Odbc+Driver+17+for+SQL+Server&trusted_connection=yes')
    print("Man, you made it!")
except Exception as e:
    print("Database not connected!, Think more you are too close")

Man, you made it!


# SCD-0

## Inserting the Values through hard coded method

In [5]:
import pandas as pd

data = {
    'Employee_Id' : ['101','102','103','104'],
    'Name' : ['John','Smith','Reg','Brik'],
    'DOB' : ['12-10-1996','13-10-1996','14-10-1996','15-10-1996'],
    'Gender':['Male','Male','Female','Female'],
    'Role': ['SDE','Data Engineer','Business Analyst','DevOps']
}

mod_data = {
    'Employee_Id' : ['101','102','103','104'],
    'Name' : ['John','Smith','Reg','Brik'],
    'DOB' : ['12-10-1996','13-11-1996','14-10-1996','15-11-1996'],
    'Gender':['Male','Female','Female','Male'],
    'Role': ['System Engineer','SDE','Business Mangager','Testing']
}

## Converting the data into Dataframe

In [6]:
dim_data= pd.DataFrame(data)
staging_data = pd.DataFrame(mod_data)

In [7]:
dim_data

,Employee_Id,Name,DOB,Gender,Role
0,101,John,12-10-1996,Male,SDE
1,102,Smith,13-10-1996,Male,Data Engineer
2,103,Reg,14-10-1996,Female,Business Analyst
3,104,Brik,15-10-1996,Female,DevOps


In [8]:
staging_data

,Employee_Id,Name,DOB,Gender,Role
0,101,John,12-10-1996,Male,System Engineer
1,102,Smith,13-11-1996,Female,SDE
2,103,Reg,14-10-1996,Female,Business Mangager
3,104,Brik,15-11-1996,Male,Testing


## Access to User Login Credentials in SQL

In [9]:
# USE SCD;

# -- Grant user access if it doesn't already exist
# IF NOT EXISTS (
#     SELECT * FROM sys.database_principals WHERE name = 'DESKTOP-4IVU8N4\Abhinay'
# )
# BEGIN
#     CREATE USER [DESKTOP-4IVU8N4\Abhinay] FOR LOGIN [DESKTOP-4IVU8N4\Abhinay];
#     ALTER ROLE db_owner ADD MEMBER [DESKTOP-4IVU8N4\Abhinay];
# END


## Insertion of Tables

In [10]:
try:
    dim_data.to_sql("Existing_Data",con=engine,if_exists='replace',index=False)
    staging_data.to_sql("Latest_Data",con=engine,if_exists='replace',index=False)
    print("Tables are inserted")
except Exception as e:
    print("Tables are not inserted, try more you will do it!",e)

Tables are inserted


## Merge upon Existing_Data and Latest_Data

In [11]:
merged_data = pd.merge(dim_data,staging_data,on="Employee_Id",suffixes=("_old","_new"))
merged_data.head()

,Employee_Id,Name_old,DOB_old,Gender_old,Role_old,Name_new,DOB_new,Gender_new,Role_new
0,101,John,12-10-1996,Male,SDE,John,12-10-1996,Male,System Engineer
1,102,Smith,13-10-1996,Male,Data Engineer,Smith,13-11-1996,Female,SDE
2,103,Reg,14-10-1996,Female,Business Analyst,Reg,14-10-1996,Female,Business Mangager
3,104,Brik,15-10-1996,Female,DevOps,Brik,15-11-1996,Male,Testing


## Verfiying the SCD_0 Type for any Violations 

In [12]:
violations_of_scd0 = merged_data[(merged_data['DOB_old']!=merged_data['DOB_new']) | (merged_data['Gender_old']!=merged_data['Gender_new'])]

if not violations_of_scd0.empty:
    print("Violation of SCD_0, Changes in Immutuable Fields\n") 
    print(violations_of_scd0[['Employee_Id','DOB_old','DOB_new','Gender_old','Gender_new']])
else:
    print("Stay Relax, the data is free from SCD_0")

Violation of SCD_0, Changes in Immutuable Fields

  Employee_Id     DOB_old     DOB_new Gender_old Gender_new
1         102  13-10-1996  13-11-1996       Male     Female
3         104  15-10-1996  15-11-1996     Female       Male


## Creation of Log File for the violation of SCD_0 Type

In [13]:
violations_of_scd0.to_csv("scd_violations_log_file.csv",mode='a',index=False)
Cleaned_Data = merged_data[~merged_data['Employee_Id'].isin(violations_of_scd0['Employee_Id'])]
Cleaned_Data

,Employee_Id,Name_old,DOB_old,Gender_old,Role_old,Name_new,DOB_new,Gender_new,Role_new
0,101,John,12-10-1996,Male,SDE,John,12-10-1996,Male,System Engineer
2,103,Reg,14-10-1996,Female,Business Analyst,Reg,14-10-1996,Female,Business Mangager


# SCD-1

## Connect to the SQL Server

In [14]:
con = pyodbc.connect(
    'DRIVER={ODBC Driver 17 for SQL Server};'
    'SERVER=DESKTOP-4IVU8N4\\MSSQLSERVER1;'
    'DATABASE=SCD;'
    'Trusted_Connection=yes;'
)

## Updating the Table Records

In [15]:
cursor = con.cursor()
insert_query="""
INSERT INTO Existing_Data(Employee_Id,Name,DOB,Gender,Role)
Values
(?,?,?,?,?)
"""
value=(105,"Jane","15-10-1996","Male","SDE")
values=(106,"Rookie","12-11-1996","Female","Data Analyst")
cursor.execute(insert_query,value)
cursor.execute(insert_query,values)
con.commit()

In [16]:
insert_query_into_latest="""
INSERT INTO Latest_Data(Employee_Id,Name,DOB,Gender,Role)
Values
(?,?,?,?,?)
"""
value_lat=(105,"Jane Smith","15-10-1996","Male","DevOps")
values_lat=(106,"Rookie Paul","12-11-1996","Female","HR")
cursor.execute(insert_query_into_latest,value_lat)
cursor.execute(insert_query_into_latest,values_lat)
con.commit()

## Reading the Table Records

In [17]:
dim_data=pd.read_sql("Select * from Existing_data",con)
dim_data

C:\Users\Abhinay\AppData\Local\Temp\ipykernel_17096\1211182508.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  dim_data=pd.read_sql("Select * from Existing_data",con)


,Employee_Id,Name,DOB,Gender,Role
0,101,John,12-10-1996,Male,SDE
1,102,Smith,13-10-1996,Male,Data Engineer
2,103,Reg,14-10-1996,Female,Business Analyst
3,104,Brik,15-10-1996,Female,DevOps
4,105,Jane,15-10-1996,Male,SDE
5,106,Rookie,12-11-1996,Female,Data Analyst


In [18]:
staging_data=pd.read_sql("Select * from Latest_Data",con)
staging_data

C:\Users\Abhinay\AppData\Local\Temp\ipykernel_17096\3706293800.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  staging_data=pd.read_sql("Select * from Latest_Data",con)


,Employee_Id,Name,DOB,Gender,Role
0,101,John,12-10-1996,Male,System Engineer
1,102,Smith,13-11-1996,Female,SDE
2,103,Reg,14-10-1996,Female,Business Mangager
3,104,Brik,15-11-1996,Male,Testing
4,105,Jane Smith,15-10-1996,Male,DevOps
5,106,Rookie Paul,12-11-1996,Female,HR


## Updating the Tables for the violation of SCD 1

In [19]:
update_query ="""
Update ed
Set
    ed.Name = ld.Name
from
    Existing_Data ed
join
    Latest_Data ld on ed.Employee_Id=ld.Employee_Id
where
    ed.Name <> ld.Name
"""
cursor.execute(update_query)
con.commit()

In [20]:
dim_data=pd.read_sql("Select * from Existing_Data",con)
dim_data

C:\Users\Abhinay\AppData\Local\Temp\ipykernel_17096\272427109.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  dim_data=pd.read_sql("Select * from Existing_Data",con)


,Employee_Id,Name,DOB,Gender,Role
0,101,John,12-10-1996,Male,SDE
1,102,Smith,13-10-1996,Male,Data Engineer
2,103,Reg,14-10-1996,Female,Business Analyst
3,104,Brik,15-10-1996,Female,DevOps
4,105,Jane Smith,15-10-1996,Male,SDE
5,106,Rookie Paul,12-11-1996,Female,Data Analyst


In [21]:
merge_data=pd.merge(dim_data,staging_data,on="Employee_Id",suffixes=('_old',"_new"))
merge_data

,Employee_Id,Name_old,DOB_old,Gender_old,Role_old,Name_new,DOB_new,Gender_new,Role_new
0,101,John,12-10-1996,Male,SDE,John,12-10-1996,Male,System Engineer
1,102,Smith,13-10-1996,Male,Data Engineer,Smith,13-11-1996,Female,SDE
2,103,Reg,14-10-1996,Female,Business Analyst,Reg,14-10-1996,Female,Business Mangager
3,104,Brik,15-10-1996,Female,DevOps,Brik,15-11-1996,Male,Testing
4,105,Jane Smith,15-10-1996,Male,SDE,Jane Smith,15-10-1996,Male,DevOps
5,106,Rookie Paul,12-11-1996,Female,Data Analyst,Rookie Paul,12-11-1996,Female,HR


## Performing the overriden process on Role Field

In [22]:
for index, row in merge_data.iterrows():
    if row['Role_old']!=row['Role_new']:
        dim_data.loc[dim_data['Employee_Id']==row['Employee_Id'],'Role']=row['Role_new']

In [23]:
dim_data

,Employee_Id,Name,DOB,Gender,Role
0,101,John,12-10-1996,Male,System Engineer
1,102,Smith,13-10-1996,Male,SDE
2,103,Reg,14-10-1996,Female,Business Mangager
3,104,Brik,15-10-1996,Female,Testing
4,105,Jane Smith,15-10-1996,Male,DevOps
5,106,Rookie Paul,12-11-1996,Female,HR


In [24]:
dim_data.to_sql("Existing_Data", con=engine, if_exists='replace', index=False)

6

In [25]:
dim_data=pd.read_sql("Select * from Existing_Data",con)
dim_data

C:\Users\Abhinay\AppData\Local\Temp\ipykernel_17096\272427109.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  dim_data=pd.read_sql("Select * from Existing_Data",con)


,Employee_Id,Name,DOB,Gender,Role
0,101,John,12-10-1996,Male,System Engineer
1,102,Smith,13-10-1996,Male,SDE
2,103,Reg,14-10-1996,Female,Business Mangager
3,104,Brik,15-10-1996,Female,Testing
4,105,Jane Smith,15-10-1996,Male,DevOps
5,106,Rookie Paul,12-11-1996,Female,HR


# SCD 2

## Pre-Defined End Date

In [26]:
safe_max_date=pd.to_datetime('2200-12-31')

## Creation of Tables through Hard Code

### Dimension Table

In [27]:
data={
    'Employee_Id':['101','102','103','104','105','106'],
    'Name' : ['John','Smith','Reg','Brik','Jane Smith','Rookie Paul'],
    'Role': ['System Engineer','SDE','Business Manager','Testing','DevOps','HR'],
    'Start_Date': pd.to_datetime(['2025-12-01','2025-11-01','2025-10-01','2025-09-01','2025-08-01','2025-07-01']),
    'End_Date':[safe_max_date]*6,
    'Active_Flag':['Yes']*6
}

### Staging Table

In [28]:
stag_data = {
    'Employee_Id':['102','106'],
    'Name' :['Smith','Rookie Paul'],
    'Role' :['HR','SDE']
}

## Conversion of Raw data into DataFrame

In [29]:
dim_status = pd.DataFrame(data)
stag_status = pd.DataFrame(stag_data)

## Sending the tables into SQL Server

In [30]:
dim_status.to_sql("Existing_Status",con=engine,if_exists='replace',index=False)
stag_status.to_sql("Latest_Status",con=engine,if_exists='replace',index=False)

2

## Extracting the Tables

In [31]:
dim_status=pd.read_sql('Select * from Existing_Status',con)
dim_status

C:\Users\Abhinay\AppData\Local\Temp\ipykernel_17096\4206214618.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  dim_status=pd.read_sql('Select * from Existing_Status',con)


,Employee_Id,Name,Role,Start_Date,End_Date,Active_Flag
0,101,John,System Engineer,2025-12-01,2200-12-31,Yes
1,102,Smith,SDE,2025-11-01,2200-12-31,Yes
2,103,Reg,Business Manager,2025-10-01,2200-12-31,Yes
3,104,Brik,Testing,2025-09-01,2200-12-31,Yes
4,105,Jane Smith,DevOps,2025-08-01,2200-12-31,Yes
5,106,Rookie Paul,HR,2025-07-01,2200-12-31,Yes


In [32]:
stag_status=pd.read_sql('Select * from Latest_Status',con)
stag_status

C:\Users\Abhinay\AppData\Local\Temp\ipykernel_17096\474767121.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  stag_status=pd.read_sql('Select * from Latest_Status',con)


,Employee_Id,Name,Role
0,102,Smith,HR
1,106,Rookie Paul,SDE


## Verifying the SCD 2 violations

In [33]:
safe_max_date=pd.to_datetime('2200-12-31')
today=pd.to_datetime('2025-06-24')

if "Start_Date" not in dim_status.columns:
    dim_status["Start_Date"]=today
if "End_Date" not in dim_status.columns:
    dim_status["End_Date"]=safe_max_date
if "Active_Flag" not in dim_status.columns:
    dim_status["Active_Flag"]="Yes"


from datetime import timedelta

for index, rows in stag_status.iterrows():
    emp_id=rows['Employee_Id']
    name=rows['Name']
    role=rows['Role']

    mark=dim_status[(dim_status['Employee_Id']==emp_id) & (dim_status['Active_Flag']=='Yes')]
    
    if not mark.empty:
        current_role=mark.iloc[0]['Role']
        if role!=current_role:
            dim_status.loc[mark.index,'End_Date']=today-timedelta(days=1)
            dim_status.loc[mark.index,'Active_Flag']='No'

            new_data ={
                'Employee_Id':emp_id,
                'Name':name,
                'Role':role,
                'Start_Date':today,
                'End_Date':safe_max_date,
                'Active_Flag':'Yes'
            }
            dim_status=pd.concat([dim_status,pd.DataFrame([new_data])], ignore_index=True)

    else:
        new_data ={
                'Employee_Id':emp_id,
                'Name':name,
                'Role':role,
                'Start_Date':today,
                'End_Date':safe_max_date,
                'Active_Flag':'Yes'
            }
        dim_status=pd.concat([dim_status,pd.DataFrame([new_data])], ignore_index=True)

dim_status.sort_values(['Employee_Id'],inplace=True)
dim_status.reset_index(drop=True,inplace=True)
dim_status
                           

,Employee_Id,Name,Role,Start_Date,End_Date,Active_Flag
0,101,John,System Engineer,2025-12-01,2200-12-31,Yes
1,102,Smith,SDE,2025-11-01,2025-06-23,No
2,102,Smith,HR,2025-06-24,2200-12-31,Yes
3,103,Reg,Business Manager,2025-10-01,2200-12-31,Yes
4,104,Brik,Testing,2025-09-01,2200-12-31,Yes
5,105,Jane Smith,DevOps,2025-08-01,2200-12-31,Yes
6,106,Rookie Paul,HR,2025-07-01,2025-06-23,No
7,106,Rookie Paul,SDE,2025-06-24,2200-12-31,Yes


# SCD 3

## Creation of Table

In [34]:
cursor=con.cursor()
create_query="""
    Create Table Existing_Position(
        Employee_Id Int,
        Name Varchar(100),
        Role Varchar(100)
    )
"""
cursor.execute(create_query)
con.commit()

ProgrammingError: ('42S01', "[42S01] [Microsoft][ODBC Driver 17 for SQL Server][SQL Server]There is already an object named 'Existing_Position' in the database. (2714) (SQLExecDirectW)")

## Insertion of Tables

In [ ]:
insert_into_query="""
    Insert into Existing_Position(Employee_Id,Name,Role)
    values(?,?,?)
"""
values=('101','Abhi','SDE-1')
values_1=('102','Varshini','Data Analyst')
values_2=('103','Bharath','Front-End')
values_3=('104','Pranav','Tester')
values_4=('105','Sandeep','Business Development')
cursor.execute(insert_into_query,values)
cursor.execute(insert_into_query,values_1)
cursor.execute(insert_into_query,values_2)
cursor.execute(insert_into_query,values_3)
cursor.execute(insert_into_query,values_4)
cursor.commit()
con.commit()

In [ ]:
dim_pos=pd.read_sql("Select * from Existing_Position",con)
dim_pos

C:\Users\Abhinay\AppData\Local\Temp\ipykernel_11940\4007044115.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  dim_pos=pd.read_sql("Select * from Existing_Position",con)


,Employee_Id,Name,Role
0,101,Abhi,SDE-1
1,102,Varshini,Data Analyst
2,103,Bharath,Front-End
3,104,Pranav,Tester
4,105,Sandeep,Business Development


In [ ]:
create_query="""
    Create Table Latest_Position(
        Employee_Id Int,
        Name Varchar(100),
        Role Varchar(100),
        Update_Date date
    )
"""
cursor.execute(create_query)
con.commit()

In [ ]:
today=pd.to_datetime('2025-06-25')
cursor=con.cursor()
insert_into_lat_pos="""
    Insert into Latest_Position(Employee_Id,Name,Role,Update_Date)
    Values(?,?,?,?)
"""
value=(101,'Abhinay','SDE-2',today)
value_1=(103,'Bharath','Full_Stack_Developer',today)

cursor.execute(insert_into_lat_pos,value)
cursor.execute(insert_into_lat_pos,value_1)
con.commit()

In [ ]:
stag_pos=pd.read_sql("Select * from Latest_Position",con)
stag_pos

C:\Users\Abhinay\AppData\Local\Temp\ipykernel_11940\2137588570.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  stag_pos=pd.read_sql("Select * from Latest_Position",con)


,Employee_Id,Name,Role,Update_Date
0,101,Abhinay,SDE-2,2025-06-25
1,103,Bharath,Full_Stack_Developer,2025-06-25


# Verfying the SCD-3 Violation

In [ ]:
from datetime import date, timedelta
today = date.today()

if 'Previous_Role' not in dim_pos.columns:
    dim_pos['Previous_Role']=None
if 'Last_Update' not in dim_pos.columns:
    dim_pos['Last_Update']=today-timedelta(days=1)
    
for index, rows in stag_pos.iterrows():
    empid=rows['Employee_Id']
    name_scd3=rows['Name']
    role=rows['Role']
    update_date=rows['Update_Date']

    if empid in dim_pos['Employee_Id'].values:
        current_role=dim_pos.loc[dim_pos['Employee_Id']==empid,'Role'].values[0]
    if current_role!=role:
        dim_pos.loc[dim_pos['Employee_Id']==empid,'Previous_Role']=current_role
        dim_pos.loc[dim_pos['Employee_Id']==empid,'Role']=role
        dim_pos.loc[dim_pos['Employee_Id']==empid,'Last_Update']=update_date
        
    else:
        new_pos={
            'Employee_Id':empid,
            'Name':name_scd3,
            'Role':role,
            'Last_Update':update_date

        }
    pd.concat([dim_pos,pd.DataFrame([new_pos])],ignore_index=True)

## Rearranging the columns

In [ ]:
column_order=['Employee_Id','Name','Role','Previous_Role','Last_Update']
dim_pos=dim_pos[column_order]
dim_pos

,Employee_Id,Name,Role,Previous_Role,Last_Update
0,101,Abhi,SDE-2,SDE-1,2025-06-25
1,102,Varshini,Data Analyst,None,2025-06-24
2,103,Bharath,Full_Stack_Developer,Front-End,2025-06-25
3,104,Pranav,Tester,None,2025-06-24
4,105,Sandeep,Business Development,None,2025-06-24


# SCD 4 

In [ ]:
df=pd.read_csv("order_data 1.csv")

## Sending the data into SQL

In [ ]:
df.to_sql("Current_Orders",con=engine,if_exists='replace',index=False)

255

In [ ]:
dim_cur_orders=pd.read_sql("Select * from Current_Orders",con=engine)
dim_cur_orders

,order_id,customer_id,order_date,order_amount,order_status,product_category
0,892a07a4-d252-4775-85e0-73077143e1c6,966,2024-11-27,317.64,Cancelled,Home & Garden
1,ae160758-e187-47b2-9350-032f88f55491,345,2023-03-27,645.87,Completed,Home & Garden
2,7c50456e-6123-45cc-aa19-128bef3754d6,503,2024-03-31,880.86,Pending,Clothing
3,c788b56b-3716-4cd9-a827-d4dc401ba00c,385,2023-08-09,876.83,Cancelled,Home & Garden
4,925ab5b1-adb1-4302-a70b-1c2db724e02b,817,2023-04-19,264.53,Pending,Home & Garden
...,...,...,...,...,...,...
1995,cb6f0f6b-3d4b-402c-ba9e-a8e666422d97,799,2023-11-20,634.62,Cancelled,Clothing
1996,900fdea4-fd87-486d-bc20-26d045767ed8,692,2023-07-20,945.97,Cancelled,Electronics
1997,4550ccfe-18c1-45d5-8c31-0edf1e3d9fa3,695,2024-01-06,229.99,Cancelled,Home & Garden
1998,dd19b028-dfa2-4eda-a66f-44f5401ef27e,502,2023-02-05,596.65,Completed,Electronics


## Creating a Historical Table 

In [ ]:
dim_his_orders=pd.DataFrame(columns=['order_id','customer_id','order_date','order_amount','order_status','product_category','Version'])

## Creating a Staging Table

In [ ]:
data=pd.DataFrame({
    'order_id':['2e76eb6f-ff16-4948-a1a0-67ac69172649','070eeb51-487b-4935-9550-75aac363ebb5','070eeb51-487b-4935-9550-75aac363ebb5','ed82969d-edf5-400c-ae65-c6d172cf1f73','ed82969d-edf5-400c-ae65-c6d172cf1f73'],
    'customer_id':['8','17','17','764','764'],
    'order_date':pd.to_datetime(['16-05-2024','17-10-2024','17-10-2024','01-01-2023','01-01-2023']),
    'order_amount':['196.27','930.65','930.65','971.86','971.86'],
    'order_status':['Completed','Cancelled','Pending','Completed','Returned'],
    'product_category':['Clothing','Books','Skincare','Food','Home & Garden']
})

C:\Users\Abhinay\AppData\Local\Temp\ipykernel_6988\547008414.py:4: UserWarning: Parsing dates in %d-%m-%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  'order_date':pd.to_datetime(['16-05-2024','17-10-2024','17-10-2024','01-01-2023','01-01-2023']),


## Sending the Table into SQL Server


In [ ]:
data.to_sql("Latest_Orders",con=engine,index=False,if_exists='replace')

5

In [ ]:
stag_orders=pd.read_sql("Select * from Latest_Orders",con=engine)
stag_orders

,order_id,customer_id,order_date,order_amount,order_status,product_category
0,2e76eb6f-ff16-4948-a1a0-67ac69172649,8,2024-05-16,196.27,Completed,Clothing
1,070eeb51-487b-4935-9550-75aac363ebb5,17,2024-10-17,930.65,Cancelled,Books
2,070eeb51-487b-4935-9550-75aac363ebb5,17,2024-10-17,930.65,Pending,Skincare
3,ed82969d-edf5-400c-ae65-c6d172cf1f73,764,2023-01-01,971.86,Completed,Food
4,ed82969d-edf5-400c-ae65-c6d172cf1f73,764,2023-01-01,971.86,Returned,Home & Garden


## Performing the SCD 4 and verification

In [ ]:
for index,rows in stag_orders.iterrows():
    orderid=rows['order_id']
    customerid=rows['customer_id']
    orderdate=rows['order_date']
    orderamount=rows['order_amount']
    orderstatus=rows['order_status']
    category=rows['product_category']

    exisitng=dim_cur_orders[dim_cur_orders['order_id']==orderid]

    if exisitng.empty:
        new={
            'order_id':orderid,
            'customer_id':customerid,
            'order_date':orderdate,
            'order_amount':orderamount,
            'order_status':orderstatus,
            'product_category':category
        }
        dim_cur_orders=pd.concat([dim_cur_orders,pd.DataFrame([new])],ignore_index=True)

    else:
        current_order_date=exisitng.iloc[0]['order_date']
        current_order_status=exisitng.iloc[0]['order_status']
        current_product_category=exisitng.iloc[0]['product_category']

        if (current_order_date!=orderdate or current_order_status!=orderstatus) or current_product_category!=category:
            prev_version=dim_his_orders[dim_his_orders['order_id']==orderid]['Version'].max()
            current_version=1 if pd.isna(prev_version) else int(prev_version)+1

            history_data={
            'order_id':orderid,
            'customer_id':customerid,
            'order_date':orderdate,
            'order_amount':orderamount,
            'order_status':orderstatus,
            'product_category':category,
            'Version':current_version
            }

            history_row = exisitng.to_dict()
            history_row['Version'] = current_version

            dim_his_orders=pd.concat([dim_his_orders,pd.DataFrame([history_data])],ignore_index=True)

        dim_cur_orders.loc[dim_cur_orders['order_id']==orderid,['order_date','order_status','product_category']]=[orderdate,orderstatus,category]
                                                                                                                            
dim_his_orders   

C:\Users\Abhinay\AppData\Local\Temp\ipykernel_6988\2789495861.py:44: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  dim_his_orders=pd.concat([dim_his_orders,pd.DataFrame([history_data])],ignore_index=True)


,order_id,customer_id,order_date,order_amount,order_status,product_category,Version
0,2e76eb6f-ff16-4948-a1a0-67ac69172649,8,2024-05-16,196.27,Completed,Clothing,1
1,070eeb51-487b-4935-9550-75aac363ebb5,17,2024-10-17,930.65,Cancelled,Books,1
2,070eeb51-487b-4935-9550-75aac363ebb5,17,2024-10-17,930.65,Pending,Skincare,2
3,ed82969d-edf5-400c-ae65-c6d172cf1f73,764,2023-01-01,971.86,Completed,Food,1
4,ed82969d-edf5-400c-ae65-c6d172cf1f73,764,2023-01-01,971.86,Returned,Home & Garden,2


## Verifying the current_order changes

In [ ]:
dim_cur_orders[dim_cur_orders["customer_id"]==764]

,order_id,customer_id,order_date,order_amount,order_status,product_category
1169,e6f759ed-f648-4537-8a6a-358005239e3b,764,2023-05-05,618.29,Pending,Toys
1224,ed82969d-edf5-400c-ae65-c6d172cf1f73,764,2023-01-01 00:00:00,971.86,Returned,Home & Garden


## Updating them into the SQL Server

In [ ]:
dim_cur_orders.to_sql("Current_Orders",con=engine,if_exists='replace',index=False)

255

In [ ]:
dim_his_orders.to_sql("Historical_Orders",con=engine,if_exists='replace',index=False)

5

# SCD 6

## Creating Tables for Dimension Data

In [35]:
cursor=con.cursor()
create_table_query="""
Create Table Existing_Info(
    Employee_Id Int Primary key,
    Name Varchar(75),
    Email Varchar(50),
    Start_Date date,
    End_Date date,
    Current_Role Varchar(30),
    Active_Flag Varchar(10)
)
"""
cursor.execute(create_table_query)

In [36]:
safe_max_date=pd.to_datetime('2200-12-31')
insert_into_table="""
Insert into Existing_Info(Employee_Id,Name,Email,Start_Date,End_Date,Current_Role,Active_Flag)
Values(?,?,?,?,?,?,?)
"""
value=(101,'A','a@gmail.com',pd.to_datetime('2025-01-01'),safe_max_date,'SDE-Intern','Yes')
value1=(102,'B','b@gmail.com',pd.to_datetime('2025-02-01'),safe_max_date,'DataEngineer-Intern','Yes')
value2=(103,'C','c@gmail.com',pd.to_datetime('2025-03-01'),safe_max_date,'ML Engineer-Intern','Yes')
value3=(104,'D','d@gmail.com',pd.to_datetime('2025-04-01'),safe_max_date,'Tester-Intern','Yes')
value4=(105,'E','f@gmail.com',pd.to_datetime('2025-05-01'),safe_max_date,'HR-Intern','Yes')
cursor.execute(insert_into_table,value)
cursor.execute(insert_into_table,value1)
cursor.execute(insert_into_table,value2)
cursor.execute(insert_into_table,value3)
cursor.execute(insert_into_table,value4)
con.commit()

In [37]:
dim_info=pd.read_sql('Select * from Existing_Info',con=engine)

In [38]:
dim_info

,Employee_Id,Name,Email,Start_Date,End_Date,Current_Role,Active_Flag
0,101,A,a@gmail.com,2025-01-01,2200-12-31,SDE-Intern,Yes
1,102,B,b@gmail.com,2025-02-01,2200-12-31,DataEngineer-Intern,Yes
2,103,C,c@gmail.com,2025-03-01,2200-12-31,ML Engineer-Intern,Yes
3,104,D,d@gmail.com,2025-04-01,2200-12-31,Tester-Intern,Yes
4,105,E,f@gmail.com,2025-05-01,2200-12-31,HR-Intern,Yes


## Creating Stag Data

In [39]:
data_creation=pd.DataFrame({
    'Employee_Id':[101,103,105,101],
    'Name':['A','C','E','A'],
    'Email':['a@gmail.com','c@gmail.com','e@gmail.com','a@gmail.com'],
    'Start_Date':pd.to_datetime(['2025-03-01','2025-05-01','2025-06-01','2025-05-01']),
    'Role':['SDE-1','ML Engineer','HR','SDE-2']
})

In [40]:
stag_info=data_creation.to_sql("Latest_Info",con=engine,if_exists='replace',index=False)

In [41]:
stag_info=pd.read_sql("Select * from Latest_Info",con=engine)

In [42]:
stag_info

,Employee_Id,Name,Email,Start_Date,Role
0,101,A,a@gmail.com,2025-03-01,SDE-1
1,103,C,c@gmail.com,2025-05-01,ML Engineer
2,105,E,e@gmail.com,2025-06-01,HR
3,101,A,a@gmail.com,2025-05-01,SDE-2


## Verifying the SCD 6 Violations

In [ ]:
from datetime import date,timedelta

if 'End_Date' not in dim_info.columns:
    dim_info["End_Date"]=pd.NaT
if "Active_Flag" not in dim_info.columns:
    dim_info["Active_Flag"]="Yes"
if 'Previous_Role' not in dim_info.columns:
    dim_info['Previous_Role']=pd.NA

for index,rows in stag_info.iterrows():
    empid=rows['Employee_Id']
    name=rows['Name']
    email=rows['Email']
    startdate=rows['Start_Date']
    role=rows['Role']

    existing=dim_info.loc[(dim_info['Employee_Id']==empid) & (dim_info['Active_Flag']=='Yes')]

    if existing.empty:
        new_data={
            'Employee_Id':empid,
            'Name':name,
            'Email':email,
            'Start_Date':startdate,
            'End_Date':safe_max_date,
            'Current_Role':role,
            'Previous_Role':pd.NA,
            'Active_Flag':'Yes'
        }
        dim_info=pd.concat([dim_info,pd.DataFrame([new_data])],ignore_index=True)
    else:
        current_role=existing.iloc[0]['Current_Role']
        if current_role!=role:
            #----------------- SCD 1,2,3 Type --------------------
            dim_info.loc[existing.index,'Email']=email
            dim_info.loc[existing.index,'End_Date']=startdate-timedelta(days=1)
            dim_info.loc[existing.index,'Active_Flag']='No'

            newdata={
                'Employee_Id':empid,
                'Name':name,
                'Email':email,
                'Start_Date':startdate,
                'End_Date':safe_max_date,
                'Current_Role':role,
                'Previous_Role':current_role,
                'Active_Flag':'Yes'
            }
            dim_info=pd.concat([dim_info,pd.DataFrame([newdata])],ignore_index=True)

for col in ["Start_Date","End_Date"]:
    dim_info[col]=pd.to_datetime(dim_info[col]).dt.date

## Reframing the Columns Accordingly

In [44]:
columns_order=['Employee_Id','Name','Email','Start_Date','End_Date','Current_Role','Previous_Role','Active_Flag']
dim_info=dim_info[columns_order]
dim_info.sort_values(["Employee_Id","Start_Date"]).reset_index(drop=True)

,Employee_Id,Name,Email,Start_Date,End_Date,Current_Role,Previous_Role,Active_Flag
0,101,A,a@gmail.com,2025-01-01,2025-02-28,SDE-Intern,NaN,No
1,101,A,a@gmail.com,2025-03-01,2025-04-30,SDE-1,SDE-Intern,No
2,101,A,a@gmail.com,2025-05-01,2200-12-31,SDE-2,SDE-1,Yes
3,102,B,b@gmail.com,2025-02-01,2200-12-31,DataEngineer-Intern,NaN,Yes
4,103,C,c@gmail.com,2025-03-01,2025-04-30,ML Engineer-Intern,NaN,No
5,103,C,c@gmail.com,2025-05-01,2200-12-31,ML Engineer,ML Engineer-Intern,Yes
6,104,D,d@gmail.com,2025-04-01,2200-12-31,Tester-Intern,NaN,Yes
7,105,E,e@gmail.com,2025-05-01,2025-05-31,HR-Intern,NaN,No
8,105,E,e@gmail.com,2025-06-01,2200-12-31,HR,HR-Intern,Yes
